In [39]:
import pandas as pd
df = pd.read_csv("/home/capeie/covid_severity_predictor/final_translated.csv")

In [40]:
df.columns

Index(['Unnamed: 0', 'Gender', 'Hospitalization', 'Death', 'Ventilator',
       'Pneumonia', 'Age', 'Pregnant', 'Diabetes', 'Asthma',
       'Immunosuppression', 'Hypertension', 'Cardiovascular Disease',
       'Obesity ', 'Chronic Renal Insufficiency', 'Tobacco Use',
       'Contact COVID', 'ICU'],
      dtype='object')

In [41]:
df = df.drop(["ICU","Ventilator","Death","Unnamed: 0"],axis=1)

In [42]:
target = df['Hospitalization']

In [43]:
x_values = df.drop(['Hospitalization'],axis=1)
x_values = x_values.sort_index(axis=1)


In [44]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()


In [45]:
x_values['Age'] = min_max_scaler.fit_transform(x_values[['Age']])


In [46]:
x_values.head()


,Age,Asthma,Cardiovascular Disease,Chronic Renal Insufficiency,Contact COVID,Diabetes,Gender,Hypertension,Immunosuppression,Obesity,Pneumonia,Pregnant,Tobacco Use
0,0.765306,2,2,2,2,1,1,2,2,2,1,2,2
1,0.224490,2,2,2,1,2,2,2,2,2,2,97,2
2,0.408163,2,2,2,2,2,2,2,2,2,1,97,2
3,0.295918,2,2,2,1,2,1,2,2,2,2,2,2
4,0.622449,2,2,2,1,1,1,2,2,1,2,2,2


In [47]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

In [48]:
#testing
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)

In [49]:
x_values.head()

,Age,Asthma,Cardiovascular Disease,Chronic Renal Insufficiency,Contact COVID,Diabetes,Gender,Hypertension,Immunosuppression,Obesity,Pneumonia,Pregnant,Tobacco Use
0,0.765306,2,2,2,2,1,1,2,2,2,1,2,2
1,0.224490,2,2,2,1,2,2,2,2,2,2,97,2
2,0.408163,2,2,2,2,2,2,2,2,2,1,97,2
3,0.295918,2,2,2,1,2,1,2,2,2,2,2,2
4,0.622449,2,2,2,1,1,1,2,2,1,2,2,2


In [50]:
#testing
ohe.fit_transform(x_values[['Pregnant']])

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]])

In [51]:
x_values['Obesity ']


0       2
1       2
2       2
3       2
4       1
       ..
5842    2
5843    2
5844    2
5845    1
5846    2
Name: Obesity , Length: 5847, dtype: int64

In [52]:
from sklearn.compose import make_column_transformer
column_trans = make_column_transformer((OneHotEncoder(),['Gender','Pneumonia','Pregnant','Diabetes','Asthma','Immunosuppression','Hypertension','Cardiovascular Disease','Obesity ','Chronic Renal Insufficiency','Tobacco Use','Contact COVID']),remainder='passthrough')

In [53]:
column_trans.fit_transform(x_values)

array([[1.        , 0.        , 1.        , ..., 1.        , 0.        ,
        0.76530612],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.2244898 ],
       [0.        , 1.        , 1.        , ..., 1.        , 0.        ,
        0.40816327],
       ...,
       [1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.32653061],
       [1.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.55102041],
       [1.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.67346939]])

In [54]:
#making a noob model for cross reference 
x_noob = x_values.loc[:,['Age']]
y_noob = df.Hospitalization

In [55]:
y_noob.shape

(5847,)

In [56]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
classifier.fit(x_noob, y_noob)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=42)

In [57]:
from sklearn.model_selection import cross_val_score
cross_val_score(classifier,x_noob,y_noob,cv=5,scoring='accuracy').mean()


0.6784700196676245

In [58]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(column_trans,classifier)

In [59]:
from sklearn.pipeline import Pipeline


In [60]:
rfc_pipeline = Pipeline(steps = [
    ('data_preprocessing', column_trans),
    ('model', RandomForestClassifier(max_depth = 10,
                                     min_samples_leaf = 3,
                                     min_samples_split = 4,
                                     n_estimators = 200))
])

In [61]:
#train,test,val split
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_values, y_noob, random_state = 42)

In [62]:
rfc_pipeline.fit(x_train, y_train)


Pipeline(steps=[('data_preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Gender', 'Pneumonia',
                                                   'Pregnant', 'Diabetes',
                                                   'Asthma',
                                                   'Immunosuppression',
                                                   'Hypertension',
                                                   'Cardiovascular Disease',
                                                   'Obesity ',
                                                   'Chronic Renal '
                                                   'Insufficiency',
                                                   'Tobacco Use',
                                                   'Contact COVID'])])),
   

In [63]:
import joblib
joblib.dump(rfc_pipeline, '/home/capeie/covid_severity_predictor/models/rfc_pipeline_hospitalization.pkl')


['/home/capeie/covid_severity_predictor/models/rfc_pipeline_hospitalization.pkl']

In [64]:
import joblib
loaded_model = joblib.load('/home/capeie/covid_severity_predictor/models/rfc_pipeline_hospitalization.pkl')


In [65]:
pipeline_preds = loaded_model.predict(x_val)


In [66]:
pipeline_preds

array([1, 2, 2, ..., 1, 1, 2])

In [67]:
#accuracy calculation
cross_val_score(rfc_pipeline,x_val,y_val,cv=5,scoring='accuracy').mean()


0.8467974192341858

In [68]:
x_new = x_values.sample(1,random_state=99)

In [69]:
pipeline_preds = loaded_model.predict(x_new)


In [70]:
type(x_new)

pandas.core.frame.DataFrame

In [71]:
prediction_translate = {1:'Ambulatory',2:'Hospitalized'}

In [72]:
prediction_translate[pipeline_preds[0]]

'Hospitalized'

In [74]:
x_new.columns

Index(['Age', 'Asthma', 'Cardiovascular Disease',
       'Chronic Renal Insufficiency', 'Contact COVID', 'Diabetes', 'Gender',
       'Hypertension', 'Immunosuppression', 'Obesity ', 'Pneumonia',
       'Pregnant', 'Tobacco Use'],
      dtype='object')